In [42]:
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import LabelEncoder

In [43]:
train_goods = pd.concat([
    pd.read_csv('./训练集/traindata_goodsid/part-00000', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00001', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00002', header=None, names=['goods_id', 'cat_id', 'brandsn'])
], axis=0)

train_user = pd.concat([
    pd.read_csv(x, header=None, names=['user_id', 'goods_id', 'is_clk', 'is_like', 'is_addcart', 'is_order', 'expose_start_time', 'dt'], nrows=500000)
    for x in glob.glob('./训练集/traindata_user/part*')
], axis=0)

In [44]:
testa_goods = pd.concat([
    pd.read_csv('./测试集a/predict_goods_id/part-00000', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./测试集a/predict_goods_id/part-00001', header=None, names=['goods_id', 'cat_id', 'brandsn']),
], axis=0)

testa_user = pd.read_excel('./测试集a/a榜需要预测的uid_5000.xlsx')

In [45]:
user_encode = LabelEncoder()
user_encode.fit(list(train_user['user_id']) + list(train_user['user_id']))

goods_encode = LabelEncoder()
goods_encode.fit(list(train_user['goods_id']) + list(train_goods['goods_id']) + list(train_goods['goods_id']))

LabelEncoder()

In [46]:
np.mean(testa_user['user_id'].isin(train_user['user_id'])), np.mean(testa_goods['goods_id'].isin(train_goods['goods_id']))

(1.0, 0.9999751455447659)

In [47]:
train_user['user_id'].nunique(), train_user['goods_id'].nunique()

(51602, 1531577)

In [48]:
train_user.describe().round(2)

,is_clk,is_like,is_addcart,is_order,dt
count,7791816.00,7791816.00,7791816.00,7791816.00,7791816.00
mean,1.06,0.02,0.06,0.01,20230222.85
std,0.28,0.13,0.27,0.09,26.21
min,1.00,0.00,0.00,0.00,20230203.00
25%,1.00,0.00,0.00,0.00,20230209.00
50%,1.00,0.00,0.00,0.00,20230216.00
75%,1.00,0.00,0.00,0.00,20230224.00
max,121.00,22.00,28.00,15.00,20230303.00


In [49]:
train_user['user_id'].value_counts()

94f8b77847a017324845b26592fdd964    15112
b83033e19eabcdd63ecc5d1cab156916     7913
ff9aa8c43066095cc67a5568ba5fb1c5     6453
485fbc41750e44e308f30683e4c8bbff     5582
c86e5bc3780bdb839b676d7505d15a49     5311
                                    ...  
f59919d2080053a4dfc516f200e4acce        1
c69ef88929430b67468a02abf73c8df3        1
bdf8350900cc5c08ce6753753213c00c        1
3127fb685010c92499d3a98455db440b        1
e305e481199cc9881f05ae7399c76463        1
Name: user_id, Length: 51602, dtype: int64

In [50]:
train_user.loc[(train_user['user_id'] == '71e1a59e90bc7174cf6349761217c627') & (train_user['goods_id'] == '47382b8a57e5b73bdba51de5c230fded')]

,user_id,goods_id,is_clk,is_like,is_addcart,is_order,expose_start_time,dt
26,71e1a59e90bc7174cf6349761217c627,47382b8a57e5b73bdba51de5c230fded,5,0,2,1,2023-02-19 06:58:28,20230219
20236,71e1a59e90bc7174cf6349761217c627,47382b8a57e5b73bdba51de5c230fded,1,0,0,0,2023-02-22 09:28:02,20230222


In [51]:
train_data = pd.merge(train_user.iloc[:], train_goods.iloc[:], on='goods_id')

In [52]:
train_data['cat_id'].nunique(), train_data['brandsn'].nunique()

(1616, 4711)

In [53]:
train_agg_feat = train_data.loc[
    (train_data['is_order'] == 0) & (train_data['is_addcart'] != 0)
]

train_agg_feat = train_agg_feat[train_agg_feat['user_id'].isin(testa_user['user_id'])]
train_agg_feat = train_agg_feat[train_agg_feat['goods_id'].isin(testa_goods['goods_id'])]

In [54]:
train_agg_feat[['user_id', 'goods_id']].to_csv('u2i.csv', index=None)

In [55]:
train_agg_feat = train_data.iloc[:].groupby(['user_id', 'goods_id']).agg({
    'is_clk': ['sum', 'max'],
    'is_like': ['sum', 'max'],
    'is_addcart': ['sum', 'max'],
    'is_order': ['sum', 'max'],
})

train_agg_feat = train_agg_feat.reset_index()
train_agg_feat.columns = [
    'user_id',
    'goods_id',
    'is_clk_sum',
    'is_clk_max',
    'is_like_sum',
    'is_like_max',
    'is_addcart_sum',
    'is_addcart_max',
    'is_order_sum',
    'is_order_max'
 ]

In [56]:
test_goods_id_agg = train_agg_feat.groupby('goods_id').agg({
    'is_clk_sum': 'sum',
    'is_order_max': 'sum'
})
test_goods_id_agg = test_goods_id_agg[test_goods_id_agg['is_clk_sum'] > 100]
test_valid_goods = test_goods_id_agg.index

In [57]:
train_feat_downsmaple = pd.concat([
    train_agg_feat[train_agg_feat['is_order_max'] !=0],
    train_agg_feat[train_agg_feat['is_order_max'] ==0].sample(int(0.03 * len(train_agg_feat)))
], axis=0)

In [58]:
1 - train_feat_downsmaple['is_order_max'].mean()

0.7795159078177217

In [59]:
1 - train_feat_downsmaple['is_addcart_max'].mean()

0.7585836536789847

In [60]:
train_feat_downsmaple['user_id'] = user_encode.transform(train_feat_downsmaple['user_id'])
train_feat_downsmaple['goods_id'] = goods_encode.transform(train_feat_downsmaple['goods_id'])

In [61]:
import paddle
import paddle.nn as nn
from paddle.io import Dataset

class SelfDefinedDataset(Dataset):
    def __init__(self, df, mode = 'train'):
        super(SelfDefinedDataset, self).__init__()
        self.df = df
        self.mode = mode

    def __getitem__(self, idx):
        if self.mode == 'predict':
            return (
                self.df['user_id'].iloc[idx],
                self.df['goods_id'].iloc[idx],
                self.df['is_clk_max'].iloc[idx], self.df['is_like_max'].iloc[idx],
                self.df['is_addcart_max'].iloc[idx],
            )
        else:
            return (
                self.df['user_id'].iloc[idx],
                self.df['goods_id'].iloc[idx],
                self.df['is_clk_max'].iloc[idx],
                (self.df['is_like_max'].iloc[idx]!= 0).astype(int),
                (self.df['is_addcart_max'].iloc[idx] != 0).astype(int),
                (self.df['is_order_max'].iloc[idx] != 0).astype(int)
            )

    def __len__(self):
        return len(self.df)

In [62]:
train_feat_downsmaple['is_order_max'].max()

15

In [63]:
train_feat_downsmaple = train_feat_downsmaple.sample(frac=1.0)

traindataset = SelfDefinedDataset(train_feat_downsmaple.iloc[:-int(-0.2*len(train_feat_downsmaple))])
train_loader = paddle.io.DataLoader(traindataset, batch_size = 128, shuffle = True)

validdataset = SelfDefinedDataset(train_feat_downsmaple.iloc[-int(-0.2*len(train_feat_downsmaple)):])
valid_loader = paddle.io.DataLoader(validdataset, batch_size = 128, shuffle = True)

In [64]:
EMBEDDING_SIZE = 256

# 定义深度学习模型
class RecommenderNet(nn.Layer):
    def __init__(self, num_users, num_goods, embedding_size, numeric_size):
        super(RecommenderNet, self).__init__()
        self.num_users = num_users
        self.num_goods = num_goods
        self.embedding_size = embedding_size

        weight_attr_user = paddle.ParamAttr(
            regularizer = paddle.regularizer.L2Decay(1e-6),
            initializer = nn.initializer.KaimingNormal()
            )
        self.user_embedding = nn.Embedding(
            num_users,
            embedding_size,
            # weight_attr=weight_attr_user
        )

        weight_attr_movie = paddle.ParamAttr(
            regularizer = paddle.regularizer.L2Decay(1e-6),
            initializer = nn.initializer.KaimingNormal()
            )
        self.goods_embedding = nn.Embedding(
            num_goods,
            embedding_size,
            # weight_attr=weight_attr_movie
        )

        self.linear = nn.Linear(
            2 * embedding_size, 2
        )

    def forward(self, data):
        user, goods, feat = data[0], data[1], [data[idx] for idx in [2, 3,4]]
        feat = paddle.stack(feat, 1).astype(paddle.float32)

        user_vector = self.user_embedding(user)
        goods_vector = self.goods_embedding(goods)
        x = paddle.concat([user_vector, goods_vector], 1)
        return self.linear(x)

In [65]:
# 定义模型损失函数、优化器和评价指标
model = RecommenderNet(len(user_encode.classes_), len(goods_encode.classes_), 128, 3)
model

RecommenderNet(
  (user_embedding): Embedding(51602, 128, sparse=False)
  (goods_embedding): Embedding(3465659, 128, sparse=False)
  (linear): Linear(in_features=256, out_features=2, dtype=float32)
)

In [66]:
optimizer = paddle.optimizer.SGD(parameters=model.parameters(), learning_rate=0.01)
loss_fn = nn.BCEWithLogitsLoss()